In [2]:
import os 
os.environ['TP_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
import numpy as np

2023-11-06 16:22:13.985359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Define functional model
inputs = keras.Input(shape=(28,28))
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')

dense2 = keras.layers.Dense(10, activation='softmax', name='category_output')
dense3 = keras.layers.Dense(1, activation='sigmoid', name='leftright_output')

In [4]:
x = flatten(inputs)
x = dense1(x)
outputs1 = dense2(x)
outputs2 = dense3(x)

model = keras.Model(inputs=inputs, outputs=[outputs1, outputs2], name='mnist_model')

2023-11-06 16:39:18.699923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.summary()

Model: "mnist_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 flatten (Flatten)              (None, 784)          0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 128)          100480      ['flatten[0][0]']                
                                                                                                  
 category_output (Dense)        (None, 10)           1290        ['dense[0][0]']                  
                                                                                        

In [7]:
# loss and optimizer

loss1 = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loss2 = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
mertrics = ['accuracy']

losses = {
    'category_output':loss1,
    'leftright_output':loss2
}

model.compile(loss=losses, optimizer=optim, metrics=mertrics)

In [10]:
# create data with 2 labels
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
x_train, x_test = X_train/255.0, X_test/255.0

# 0=left, 1=right
y_leftright = np.zeros(y_train.shape, dtype=np.uint8)
for idx, y in enumerate(y_train):
    if y > 5:
        y_leftright[idx] = 1
print(y_train.dtype, y_train[0:20])
print(y_leftright.dtype, y_leftright[0:20])

y = {'category_output':y_train,
'leftright_output':y_leftright}

uint8 [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]
uint8 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1]


In [11]:
# Training

model.fit(x=X_train, y=y, epochs=5, batch_size=64, verbose=2)

Epoch 1/5
938/938 - 5s - loss: 5.1907 - category_output_loss: 3.5746 - leftright_output_loss: 1.6161 - category_output_accuracy: 0.8530 - leftright_output_accuracy: 0.9042 - 5s/epoch - 5ms/step
Epoch 2/5
938/938 - 3s - loss: 0.6406 - category_output_loss: 0.4686 - leftright_output_loss: 0.1719 - category_output_accuracy: 0.9028 - leftright_output_accuracy: 0.9435 - 3s/epoch - 4ms/step
Epoch 3/5
938/938 - 4s - loss: 0.4359 - category_output_loss: 0.3173 - leftright_output_loss: 0.1186 - category_output_accuracy: 0.9268 - leftright_output_accuracy: 0.9568 - 4s/epoch - 4ms/step
Epoch 4/5
938/938 - 4s - loss: 0.3365 - category_output_loss: 0.2432 - leftright_output_loss: 0.0932 - category_output_accuracy: 0.9389 - leftright_output_accuracy: 0.9655 - 4s/epoch - 4ms/step
Epoch 5/5
938/938 - 4s - loss: 0.2890 - category_output_loss: 0.2098 - leftright_output_loss: 0.0791 - category_output_accuracy: 0.9466 - leftright_output_accuracy: 0.9715 - 4s/epoch - 4ms/step


In [12]:
predictions = model.predict(X_test)
len(predictions)

313/313 [==============================] - 1s 2ms/step


2

In [13]:
predictions_category = predictions[0]
predictions_lr = predictions[1]

pr_cat = predictions_category[0:20]
pr_lr = predictions_lr[0:20]

labels_cat = np.argmax(pr_cat, axis=1)
labels_lr = np.array([1 if p >= 0.5 else 0 for p in predictions_lr])

In [14]:
print(y_test[0:20])
print(labels_cat)
print(labels_lr)

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 8 4]
[1 0 0 ... 0 0 1]
